In [46]:
from sshtunnel import SSHTunnelForwarder

AWS_IP_ADDRESS = '18.237.94.217'
AWS_USERNAME = 'luke'
SSH_KEY_PATH = '/users/luketibbott/.ssh/id_rsa'

server = SSHTunnelForwarder(
    AWS_IP_ADDRESS,
    ssh_username=AWS_USERNAME,
    ssh_pkey=SSH_KEY_PATH,
    remote_bind_address=('localhost', 5432),
)

server.start()
print(server.is_active, server.is_alive, server.local_bind_port)

True True 50627


In [47]:
from sqlalchemy import create_engine

# Postgres username, password, and database name
POSTGRES_IP_ADDRESS = 'localhost' ## This is localhost because SSH tunnel is active
POSTGRES_PORT = str(server.local_bind_port)
POSTGRES_USERNAME = 'luke'     ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = 'luke' ## CHANGE THIS TO YOUR POSTGRES PASSWORD
POSTGRES_DBNAME = 'baseball'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_IP_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)

Please complete this exercise via SQLalchemy and Jupyter notebook.

We will be working with the Lahman baseball data we uploaded to your AWS instance in class.

1. What was the total spent on salaries by each team, each year?

2. What is the first and last year played for each player? Hint: Create a new table from 'Fielding.csv'.

3. Who has played the most all star games?

4. Which school has generated the most distinct players? Hint: Create new table from 'CollegePlaying.csv'.

5. Which players have the longest career? Assume that the debut and finalGame columns comprise the start and end, respectively, of a player's career. Hint: Create a new table from 'Master.csv'. Also note that strings can be converted to dates using the DATE function and can then be subtracted from each other yielding their difference in days.

6. What is the distribution of debut months? Hint: Look at the DATE and EXTRACT functions.

7. What is the effect of table join order on mean salary for the players listed in the main (master) table? Hint: Perform two different queries, one that joins on playerID in the salary table and other that joins on the same column in the master table. You will have to use left joins for each since right joins are not currently supported with SQLalchemy.

In [48]:
import pandas as pd

In [50]:
cnx.table_names()

['allstarfull',
 'salaries',
 'schools',
 'franchises',
 'fielding',
 'schoolsplayers',
 'master']

**Question 1:** What was the total spent on salaries by each team, each year?

In [18]:
pd.read_sql_query('''SELECT teamid, yearid, AVG(s.salary)
                     FROM salaries s
                     GROUP BY 1, 2
                     ORDER BY 2 ASC''', cnx).head()

,teamid,yearid,avg
0,PHI,1985,532892.947368
1,DET,1985,517407.150000
2,MON,1985,473508.300000
3,BAL,1985,525486.909091
4,BOS,1985,435902.400000


The above table has salary information for each team by ascending year.

**Question 2:** What is the first and last year played for each player? Hint: Create a new table from 'Fielding.csv'.

In [23]:
pd.read_sql_query('''
                  SELECT playerid, MIN(yearid) min_yr, MAX(yearid) max_yr
                  FROM Fielding
                  GROUP BY 1
                  ORDER BY 2 ASC
                  ''', cnx).head()

,playerid,min_yr,max_yr
0,conefr01,1871,1871
1,fishech01,1871,1878
2,hodesch01,1871,1874
3,quinnpa02,1871,1877
4,biermch01,1871,1871


The above table has the first and last year each player played, ascending by minimum year.

**Question 3:** Who has played the most all star games?

In [36]:
pd.read_sql_query('''
                  SELECT playerid, COUNT(playerid)
                  FROM allstarfull
                  GROUP BY 1
                  ORDER BY 2 DESC
                  LIMIT 1;
                  ''', cnx)

,playerid,count
0,aaronha01,25


Looks like Hank Aaron appeared in the most baseball games.

**Question 4:** Which school has generated the most distinct players? Hint: Create new table from 'CollegePlaying.csv'.

In [43]:
pd.read_sql_query('''
                  SELECT COUNT(DISTINCT playerid), schoolid
                  FROM schoolsplayers
                  GROUP BY 2
                  ORDER BY 1 DESC
                  LIMIT 1;
                  ''', cnx)

,count,schoolid
0,102,usc


USC has generated the most distinct players.

**Question 5:** Which players have the longest career? Assume that the debut and finalGame columns comprise the start and end, respectively, of a player's career. Hint: Create a new table from 'Master.csv'. Also note that strings can be converted to dates using the DATE function and can then be subtracted from each other yielding their difference in days.

In [70]:
pd.read_sql_query('''
                  SELECT DISTINCT namegiven, finalGame - debut AS career_days
                  FROM Master
                  WHERE finalGame IS NOT NULL AND debut IS NOT NULL
                  ORDER BY 2 DESC 
                  LIMIT 10;
                  ''', cnx)

,namegiven,career_days
0,Nicholas,12862
1,James Henry,11836
2,Saturnino Orestes Armas,11492
3,Charles Timothy,11126
4,Walter Arlington,10678
5,James Thomas,10192
6,Hugh Ambrose,9954
7,John Joseph,9897
8,Lynn Nolan,9873
9,Charles Evard,9868


Above are the top 10 players by the number of days their careers spanned. Nicholas is at the top.

**Question 6:** What is the distribution of debut months? Hint: Look at the DATE and EXTRACT functions.

In [76]:
pd.read_sql_query('''
                  SELECT date_part('month', debut) AS month, COUNT(playerid)
                  FROM Master
                  WHERE date_part('month', debut) IS NOT NULL
                  GROUP BY 1
                  ORDER BY 2 DESC;
                  ''', cnx)

,month,count
0,9.0,5061
1,4.0,4711
2,5.0,2230
3,7.0,1978
4,8.0,1943
5,6.0,1893
6,10.0,308
7,3.0,41


2018-10-18 16:37:52,387| ERROR   | Socket exception: Connection reset by peer (54)
ERROR:paramiko.transport:Socket exception: Connection reset by peer (54)


Most players debuted in September, the least players debuted in March.

**Question 7:** What is the effect of table join order on mean salary for the players listed in the main (master) table? Hint: Perform two different queries, one that joins on playerID in the salary table and other that joins on the same column in the master table. You will have to use left joins for each since right joins are not currently supported with SQLalchemy.

In [105]:
df1 = pd.read_sql_query('''
                    SELECT m.playerid, AVG(s.salary) sal
                    FROM salaries s
                    LEFT JOIN Master m
                    ON s.playerid = m.playerid
                    GROUP BY 1
                    ORDER BY 2 DESC
                  ''', cnx)

In [103]:
df2 = pd.read_sql_query('''
                    SELECT m.playerid, AVG(s.salary) sal
                    FROM Master m
                    LEFT JOIN salaries s
                    ON s.playerid = m.playerid
                    GROUP BY 1
                    ORDER BY 2 DESC
                  ''', cnx)

In [106]:
df1.shape

(4668, 2)

In [107]:
df2.shape

(18354, 2)

2018-10-18 17:10:55,710| ERROR   | Socket exception: Connection reset by peer (54)
ERROR:paramiko.transport:Socket exception: Connection reset by peer (54)


When you grab the Master table first, there are many more entries in the table. This is because the left join is saying it wants everything from the Master table, and if anything in the right table matches, I want that too.